In [1]:

import cv2



import numpy as np

import moviepy.editor as mpy

from mmengine import load

from mmaction.datasets import (CenterCrop,
                               GeneratePoseTarget,
                               PoseCompact, PoseDecode,
                               Resize)

In [2]:
FONTFACE = cv2.FONT_HERSHEY_DUPLEX
FONTSCALE = 0.6
FONTCOLOR = (255, 255, 255)
BGBLUE = (0, 119, 182)
THICKNESS = 1
LINETYPE = 1

In [3]:
def add_label(frame, label, BGCOLOR=BGBLUE):
    threshold = 30
    def split_label(label):
        label = label.split()
        lines, cline = [], ''
        for word in label:
            if len(cline) + len(word) < threshold:
                cline = cline + ' ' + word
            else:
                lines.append(cline)
                cline = word
        if cline != '':
            lines += [cline]
        return lines
    
    if len(label) > 30:
        label = split_label(label)
    else:
        label = [label]
    label = ['Action: '] + label
    
    sizes = []
    for line in label:
        sizes.append(cv2.getTextSize(line, FONTFACE, FONTSCALE, THICKNESS)[0])
    box_width = max([x[0] for x in sizes]) + 10
    text_height = sizes[0][1]
    box_height = len(sizes) * (text_height + 6)
    
    cv2.rectangle(frame, (0, 0), (box_width, box_height), BGCOLOR, -1)
    for i, line in enumerate(label):
        location = (5, (text_height + 6) * i + text_height + 3)
        cv2.putText(frame, line, location, FONTFACE, FONTSCALE, FONTCOLOR, THICKNESS, LINETYPE)
    return frame

In [4]:
# get_pseudo_heatmap
keypoint_pipeline = [
    dict(type=PoseDecode),
    dict(type=PoseCompact, hw_ratio=1., allow_imgpad=True),
    dict(type=Resize, scale=(-1, 64)),
    dict(type=CenterCrop, crop_size=64),
    dict(type=GeneratePoseTarget, sigma=0.6, use_score=True, with_kp=True, with_limb=False)
]

limb_pipeline = [
    dict(type=PoseDecode),
    dict(type=PoseCompact, hw_ratio=1., allow_imgpad=True),
    dict(type=Resize, scale=(-1, 64)),
    dict(type=CenterCrop, crop_size=64),
    dict(type=GeneratePoseTarget, sigma=0.6, use_score=True, with_kp=False, with_limb=True)
]

In [5]:
from mmengine.dataset import Compose
def get_pseudo_heatmap(anno, flag='keypoint'):
    assert flag in ['keypoint', 'limb']
    pipeline = Compose(keypoint_pipeline if flag == 'keypoint' else limb_pipeline)
    return pipeline(anno)['imgs']

def vis_heatmaps(heatmaps, channel=-1, ratio=8):
    import matplotlib.cm as cm
    
    if ( 0 <= channel <= heatmaps.shape[1]-1 ):
        heatmaps = [heatmaps[x][channel] for x in range(heatmaps.shape[0])]
    else:  # draw all keypoints / limbs on the same map
        heatmaps = [np.max(x, axis=0) for x in heatmaps]
    
    h, w = heatmaps[0].shape    
    newh, neww = int(h * ratio), int(w * ratio)
    
    cmap = cm.viridis
    heatmaps = [(cmap(x)[..., :3] * 255).astype(np.uint8) for x in heatmaps]

    heatmaps = [cv2.resize(x, (neww, newh)) for x in heatmaps]
    return heatmaps

## Visualize CIIS Heatmap Volume

In [6]:
ciis_ann_file = 'data/skeleton/ciis_0s5_v3.pkl'

In [7]:
ciis_categories = ['berdiri', 'berjalan', 'berjongkok', 'merayap', 'melempar', 'membidik senapan', 
                  'membidik pistol', 'memukul', 'menendang', 'menusuk']
ciis_annos = load(ciis_ann_file)

In [8]:
# visualize pose of which clip sub split? 'xsub_train' / 'xsub_val'
sub_split = 'xsub_train'

print("Max Index is", len(ciis_annos['split'][sub_split]))

Max Index is 2024


In [9]:
# visualize pose of which clip? index in 0 - Max Index.
idx = 0
print("Selected Clip is", ciis_annos['split'][sub_split][idx])

Selected Clip is 50d_2s2_34.001_0


In [10]:
frame_dir = ciis_annos['split'][sub_split][idx]  # <video_ID>_frameID.personID_dataID

anno = [x for x in ciis_annos['annotations'] if x['frame_dir'] == frame_dir][0]

In [11]:
keypoint_heatmap = get_pseudo_heatmap(anno)
keypoint_mapvis = vis_heatmaps(keypoint_heatmap, channel=-1)
keypoint_mapvis = [add_label(f, ciis_categories[anno['label']]) for f in keypoint_mapvis]
vid = mpy.ImageSequenceClip(keypoint_mapvis, fps=12)

# CHECK ndarray.shape, RESTART KERNEL if ERROR!
print(keypoint_heatmap.shape)  # frames X kpts X width X heighs

print("\nLength of clip is", keypoint_heatmap.shape[0], "frames\n")  # frames X kpts X width X heighs
vid.ipython_display()

(4, 17, 64, 64)

Length of clip is 4 frames

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4
